In [ ]:
from methods.slab_topo import inside_topo,surface_topo
from ase.io import read
filename = ''
atoms = read(filename)
inside_sites = inside_topo(atoms,bond_len=1.65)
print("inside_sites:",len(inside_sites))
surface_sites = surface_topo(atoms,bond_len=1.65,both_surface=True)
print("surface_sites:",len(surface_sites))


In [26]:
import random
import numpy as np 
def calculate_distance(cell,point1, point2):
    #Calculated distance methods for dealing with periodicity
    a_vec = cell[0]
    b_vec = cell[1]
    dis = 10.
    neighbors = []
    for da in range(-1,2):
        for db in range(-1,2):
            neighbor_position = point1 + da * a_vec + db * b_vec
            neighbors.append(neighbor_position)
    for n in neighbors:
        if dis > np.linalg.norm(n-point2):
            dis = np.linalg.norm(n-point2)
    return dis

def select_points(points, num_points, min_distance,cell):
    selected_points = []
    
    while len(selected_points) < num_points:
        point = random.choice(points)
        if all(calculate_distance(cell,point, selected_point) >= min_distance for selected_point in selected_points):
            selected_points.append(point)
    return selected_points

In [ ]:
from ase.neighborlist import natural_cutoffs, NeighborList
import networkx as nx
def get_graph(atoms):
    #Get Periodicity Graph
    cutoffs = natural_cutoffs(atoms)
    positions = atoms.get_positions()
    cell = atoms.get_cell()
    G = nx.Graph()
    symbols = atoms.symbols                               
    G.add_nodes_from([(i, {'symbol': symbols[i]}) 
                          for i in range(len(symbols))])
    for i in range(len(positions)):
        for j in range(i + 1, len(positions)):
            distance = calculate_distance(cell,positions[i],positions[j])
            if distance < cutoffs[i]+cutoffs[j]:
                G.add_edge(i, j)
    return G